# Exploring neural networks

## Problem 4

Given the MNIST dataset classify those images that represent numbers greater or equal to 5. In other words, what is the probability that the number is >=5.

In [2]:
import numpy as np
import random
import pandas as pd

In [3]:
df_train = pd.read_csv('Gym 5/train.csv')
df_test = pd.read_csv('Gym 5/test.csv')

In [4]:
df_train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
df_features = df_train.iloc[:, 1:785]
df_label = df_train.iloc[:, 0]

X_testdf = df_test.iloc[:, 0:784]

print(X_testdf.shape)

(28000, 784)


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_cv, y_train, y_cv = train_test_split(df_features, df_label, 
                                                test_size = 0.2,
  
                                                random_state = 1212)
eighty_percent = int(df_features.shape[0] * 0.8)
twenty_percent = int(df_features.shape[0] * 0.2)
test_size = X_testdf.shape[0]

X_train = X_train.values.reshape(eighty_percent, 784) #(33600, 784)
X_cv = X_cv.values.reshape(twenty_percent, 784) #(8400, 784)

X_test = X_testdf.values.reshape(test_size, 784)

In [12]:
X_train = X_train.astype('float32'); X_cv= X_cv.astype('float32'); X_test = X_test.astype('float32')
X_train /= 255; X_cv /= 255; X_test /= 255

In [15]:
# Input Parameters
n_input = 784 # number of features
n_hidden_1 = 300
n_hidden_2 = 100
n_hidden_3 = 100
n_hidden_4 = 200
num_digits = 10

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
#         self.conv1 = nn.Conv2d(1, 6, 3)
#         self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(n_input, n_hidden_1) 
        self.fc2 = nn.Linear(n_hidden_1, n_hidden_2)
        self.fc3 = nn.Linear(n_hidden_2, n_hidden_3)
        self.fc4 = nn.Linear(n_hidden_3, n_hidden_4)
        self.fc5 = nn.Linear(n_hidden_4, num_digits)


    def forward(self, x):
        # Max pooling over a (2, 2) window
#         x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
#         x = F.max_pool2d(F.relu(self.conv2(x)), 2)
#         x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        
        x = self.fc5(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (fc1): Linear(in_features=784, out_features=300, bias=True)
  (fc2): Linear(in_features=300, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=100, bias=True)
  (fc4): Linear(in_features=100, out_features=200, bias=True)
  (fc5): Linear(in_features=200, out_features=10, bias=True)
)


In [38]:
torch.tensor(df_train.iloc[0:100, 1:].values, dtype=torch.float32)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [40]:
def torch_train_test_data(ds, split_percent=0.8):
    
    split = round(ds.shape[0] * split_percent)
       
    x_train = torch.tensor(ds.iloc[0:split, 1:].values, dtype=torch.float32)
    y_train = torch.tensor(ds.iloc[0:split, 0].values, dtype=torch.long)
      
    train_data = data.TensorDataset(x_train, y_train)

    x_test = torch.tensor(ds.iloc[split:, 1:].values, dtype=torch.float32)
    y_test = torch.tensor(ds.iloc[split:, 0].values, dtype=torch.long)
    
    test_data = data.TensorDataset(x_test, y_test)
       
    return (x_train, y_train, x_test, y_test, train_data, test_data)


x_train, y_train, x_test, y_test, train_data, test_data = torch_train_test_data(df_train, 0.8)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=4,
                                         shuffle=False, num_workers=0) 
    
test_loader = torch.utils.data.DataLoader(test_data, batch_size=4,
                                         shuffle=False, num_workers=0)


In [41]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [43]:
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        
#         print(inputs)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        print_every = round(df_train.shape[0] * 0.1)
        if i % print_every == print_every - 1:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / i))
            running_loss = 0.0

print('Finished Training')

[1,  4200] loss: 0.539
[1,  8400] loss: 0.165
[2,  4200] loss: 0.237
[2,  8400] loss: 0.112
[3,  4200] loss: 0.191
[3,  8400] loss: 0.097
[4,  4200] loss: 0.163
[4,  8400] loss: 0.087
[5,  4200] loss: 0.141
[5,  8400] loss: 0.075
[6,  4200] loss: 0.142
[6,  8400] loss: 0.066
[7,  4200] loss: 0.143
[7,  8400] loss: 0.062
[8,  4200] loss: 0.106
[8,  8400] loss: 0.051
[9,  4200] loss: 0.095
[9,  8400] loss: 0.046
[10,  4200] loss: 0.097
[10,  8400] loss: 0.059
Finished Training


In [44]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        input_data, labels = data
        # Note input data is batched, so output is batched
        outputs = net(input_data)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the %d test images: %d %%' % (x_test.size()[0],
    100 * correct / total))

Accuracy of the network on the 8400 test images: 94 %


In [92]:
import numpy as np
from PIL import Image

def compare_prediction(i):
    output = list(net(x_test[i]).detach().numpy())

    prediction = output.index(max(output))

    array = np.array((x_test[i].reshape(28,28))).astype(np.uint8)

    img = Image.fromarray(array)
    
    return (img, prediction)

In [91]:

for i in range(0, 10):
    (img, pred) = compare_prediction(i)
    print('predicted %d ' % pred)
    display(img)

predicted 0 


predicted 7 


predicted 7 


predicted 2 


predicted 2 


predicted 6 


predicted 5 


predicted 7 


predicted 8 


predicted 5 
